In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import urllib3
import requests
import time

In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
base_url = 'https://www.hockey-reference.com/teams/'

In [3]:
def get_page(url):
    page = urlopen(base_url)
    soup = BeautifulSoup(page, 'lxml')
    file = open('hockey-reference_urls.txt', 'w')
    file.write(str(soup))
    file.close()

def get_team_links(url):
    page = urlopen(url)
    soup = BeautifulSoup(page, 'lxml')

In [4]:
get_page(base_url)

In [5]:
with open('hockey-reference_urls.txt', 'r') as file:
    for line in file:
        line = line.strip()

In [6]:
page = open("hockey-reference_urls.txt", 'r')
soup = BeautifulSoup(page, "lxml")
div = soup.find('div', {'class': 'overthrow table_container'})


In [7]:
team_links = []
links = div.find_all('a')
for link in links:
    team_links.append(link.get('href'))

In [8]:
len(team_links)

31

In [9]:
def get_team_table(url):
    res = requests.get(url)
    goalie_soup = BeautifulSoup(res.content, 'lxml')
    team_name = goalie_soup.find('h1', {'itemprop': 'name'}).find_all('span')[1].text
    table = goalie_soup.find('div', {'id': 'all_goalies'}).find('table', {'id': 'goalies'}).find('tbody')
    team_stats = []
    for row in table.find_all('tr'):
        players = {}
        for element in row:
            players['Player'] = row.find('a').text
            players['Age'] = row.find('td', {'data-stat': 'age'}).text
            players['Games Played'] = row.find('td', {'data-stat': 'games_goalie'}).text
            players['Games Started'] = row.find('td', {'data-stat': 'starts_goalie'}).text
            players['Wins'] = row.find('td', {'data-stat': 'wins_goalie'}).text
            players['Losses'] = row.find('td', {'data-stat': 'losses_goalie'}).text
            players['Ties OT Losses'] = row.find('td', {'data-stat': 'ties_goalie'}).text
            players['Goals Against'] = row.find('td', {'data-stat': 'goals_against'}).text
            players['Shots Against'] = row.find('td', {'data-stat': 'shots_against'}).text
            players['Saves'] = row.find('td', {'data-stat': 'saves'}).text
            players['Save Percentage'] = row.find('td', {'data-stat': 'save_pct'}).text
            players['Goals Against Average'] = row.find('td', {'data-stat': 'goals_against_avg'}).text
            players['Shutouts'] = row.find('td', {'data-stat': 'shutouts'}).text
            players['Minutes'] = row.find('td', {'data-stat': 'min_goalie'}).text
            players['Quality Starts'] = row.find('td', {'data-stat': 'quality_starts_goalie'}).text
            players['Quality Start Percentage'] = row.find('td', {'data-stat': 'quality_start_goalie_pct'}).text
            players['Really Bad Starts'] = row.find('td', {'data-stat': 'really_bad_starts_goalie'}).text
            players['GA Pct -'] = row.find('td', {'data-stat': 'ga_pct_minus'}).text
            players['Goals Saved Above Average'] = row.find('td', {'data-stat': 'gs_above_avg'}).text
            players['Adjusted GAA'] = row.find('td', {'data-stat': 'goals_against_avg_adjusted'}).text
            players['Goalie Point Shares'] = row.find('td', {'data-stat': 'gps'}).text
            players['Team'] = team_name
        team_stats.append(players)
    return team_stats

In [10]:
team_year = get_team_table('https://www.hockey-reference.com/teams/ANA/2018.html')

In [25]:
base_url = 'https://www.hockey-reference.com'
teams = team_links
years = ['2011.html']
year_df = pd.DataFrame()
for team in teams:
    try:
        for year in years:
            url = base_url + team + year
            team_year = get_team_table(url)
            team_df = pd.DataFrame(team_year)
            year_df = pd.concat([year_df, team_df], axis=0)
            year_df.reset_index(drop=True, inplace=True)
            time.sleep(3)
    except:
        continue

In [15]:
year_df

Adjusted GAA Age GA Pct - Games Played Games Started Goalie Point Shares  \
0          2.74  26       95           43            37                 7.0   
1          2.46  22       94           40            38                 6.5   
2          3.20  29                     9             7                 0.9   
3          3.07  28       99           64            62                11.1   
4          3.24  31      108           24            20                 3.2   
5          2.83  29       94           45            40                 8.2   
6          3.13  22      102           20            20                 3.2   
7          2.94  24       89           21            21                 4.5   
8          5.00  23                     1             1                 0.0   
9          3.13  29      107           37            37                 5.4   
10         4.22  33      142           26            23                 1.1   
11         3.31  24      115           22            19                 2.5   
12         4.13  37                     4             3                 0.2   
13         2.88  31      107           52            51                 7.0   
14         3.37  28      116           34            31                 3.9   
15         2.82  31       90           58            58                12.0   
16         3.08  27      100           29            24                 4.6   
17         1.52  34                     1             0                 0.2   
18         3.36  27      101           57            57                 9.9   
19         3.04  23       92           20            13                 3.6   
20         2.91  29       91           14            12                 2.5   
21         3.25  23                     1             0                -0.1   
22         3.09  21       95           31            30                 6.3   
23         3.29  27      109           37            37                 5.3   
24         3.95  32      130           18            12                 1.2   
25         3.70  23                     4             3                 0.5   
26         3.32  32      110           43            39                 5.5   
27         3.19  32      111           48            43                 5.9   
28         2.78  23       93           54            49                 9.5   
29         3.34  31      110           37            33                 4.8   
..          ...  ..      ...          ...           ...                 ...   
60         3.17  27      101           24            21                 4.2   
61         0.00  21                     1             0                 0.2   
62         4.14  23                     1             1                 0.2   
63         3.01  27       97           54            53                10.0   
64         2.70  27       89           32            29                 6.4   
65         2.72  31       93           58            58                11.0   
66         2.40  21       83           13            13                 2.7   
67         3.36  28       98           14            11                 2.5   
68         2.69  26       97           65            65                10.8   
69         1.99  27                     8             8                 1.8   
70         3.47  28                    13             9                 0.6   
71         2.81  25       94           47            44                 8.2   
72         2.47  30       82           42            38                 8.6   
73         4.69  22                     1             0                -0.1   
74         2.46  24                     1             0                 0.0   
75         3.44  25                     3             0                 0.2   
76         2.44  29       86           61            60                12.2   
77         3.29  21      105           24            21                 3.5   
78         1.00  23                     1            

In [26]:
year_df.to_csv('2011 goalie stats.csv')